In [1]:
def read_list(text_path):
    lsit=[]
    with open('%s' % text_path, 'r', encoding="utf8") as f:  # 打开一个文件只读模式
        line = f.readlines()  # 读取文件中的每一行，放入line列表中
        for line_list in line:
            lsit.append(line_list.replace('\n',''))
    return lsit
train_x=read_list('../data/pre_data.txt')
print(train_x[0])

穿着背心的*醒来，看看手机，三点了。


In [2]:
#ban是cls的后缀
ban='_1636180246.4086099_0_0.7081_0.7007'
cuda_num = str(3)

In [3]:
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn
import torch.nn.functional as F


class fn_cls(nn.Module):
    def __init__(self):
        super(fn_cls, self).__init__()
        self.model = AutoModel.from_pretrained("bert-base-chinese")
        self.model.to(device)
        self.dropout = nn.Dropout(0.5)
        self.l1 = nn.Linear(768, 6)
#         self.l2 = nn.Linear(768, 6)

    def forward(self, x, attention_mask=None):
        outputs = self.model(x, attention_mask=attention_mask)
#         print(outputs[0])torch.Size([8, 100, 768])
#         print(outputs[1])torch.Size([8, 768])
#         print(outputs[0][:,0,:])torch.Size([8, 768])
#         print(outputs[1])torch.Size([8, 768])
        x = outputs[1]
        x = self.dropout(x)
        x = self.l1(x)
        return x

In [4]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

In [5]:
import torch
device_s = "cuda:" + cuda_num
device = torch.device(device_s if torch.cuda.is_available() else "cpu")
cls=torch.load("../data/cls"+str(ban)+".model",map_location=device)

In [6]:
def get_output(text):
    text2id = tokenizer(
        text, max_length=100, padding='max_length', truncation=True, return_tensors="pt"
    )

    input_ids=text2id["input_ids"].to(device)
    mask=text2id["attention_mask"].to(device)
    output = cls(input_ids, attention_mask=mask)

#     output=output.round()
#     output=output.int()
    output=output.tolist()
    output0=[]
    for j in output:
        s=''
        for i in j:
            s+=str(i)
            s+=','
        s=s[:-1]
        output0.append(s)
    return output0

In [7]:
text = ['*与n3：啊？','*与n3：啊？']
# 爱、乐、惊、怒、恐、哀
print(get_output(text))

['-0.005654674954712391,0.05292457714676857,0.42862987518310547,-0.0011745281517505646,0.021912939846515656,0.019521398469805717', '-0.005654674954712391,0.05292457714676857,0.42862987518310547,-0.0011745281517505646,0.021912939846515656,0.019521398469805717']


In [8]:
result=[]

i=0
batch_size=16
while(i<len(train_x)):
    if i+batch_size<=len(train_x):
        input_x=train_x[i:i+batch_size]
#         print('{}:{},len={}'.format(i,i+batch_size,batch_size),end = "\r")
    else:
        input_x=train_x[i:len(train_x)]
#         print('{}:{},len={}'.format(i,len(train_x)-1,len(train_x)-i),end = "\r")
        
    pre=get_output(input_x)
#     print(pre)
    result+=pre
    print(len(result),end = "\r")
    i+=batch_size


In [9]:
end_list=read_list('../data/end_list.txt')
print(end_list[0])
print(len(result))
print(len(end_list))

34170_0002_A_12
21376
21376


In [10]:
ff = open('../data/result'+str(ban)+'.txt', encoding='utf-8', mode='w')
ff.write('id\temotion\n')
for i in range(len(end_list)):
    ff.write(end_list[i]+'\t'+result[i])
    ff.write("\n")
ff.close()